<a href="https://colab.research.google.com/github/RicAntonio/Police_Shootings_2015-2024_Clasificacion/blob/main/Police_Shootings_2015_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disparos policiales "Tiroteos" desde 2015 a 2024

In [ ]:
import pandas as pd
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive' , force_remount = True)

# Listar archivos en 'My Drive' para verificar la ruta
print(os.listdir('/content/drive/My Drive'))

# Intenta cargar el archivo después de verificar su ubicación
data = pd.read_csv('/content/drive/My Drive/Police_shootings_2015_2024.csv')

# Verificar las columnas disponibles en el DataFrame
print(data.columns)

Mounted at /content/drive
['Colab Notebooks', 'Pepsico_Data.csv', 'Pepsico_Project', 'Police_shootings_2015_2024', 'Police_shootings_2015_2024.csv', 'processed_shooting_data.csv']
Index(['date', 'name', 'age', 'gender', 'armed', 'race', 'city', 'state',
       'flee', 'body_camera', 'signs_of_mental_illness',
       'police_departments_involved'],
      dtype='object')


In [ ]:
import pandas as pd
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Intenta cargar el archivo después de verificar su ubicación
data = pd.read_csv('/content/drive/My Drive/Police_shootings_2015_2024.csv')

# Verificar las columnas disponibles en el DataFrame
print(data.columns)

# Supongamos que la columna correcta es 'outcome' en lugar de 'status'
# Preprocesamiento de datos
data['date'] = pd.to_datetime(data['date'])
data['age'].fillna(data['age'].median(), inplace=True)

# Ingeniería de características (ajusta los nombres de columna según corresponda)
data['is_male'] = data['gender'].apply(lambda x: 1 if x == 'M' else 0)
data['is_fatal'] = data['armed'].apply(lambda x: 1 if x == 'Fatal' else 0)  # Asegúrate de usar el nombre correcto de la columna

# Guardar las características procesadas y el objetivo en Google Drive
data.to_csv('/content/drive/My Drive/processed_shooting_data.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['date', 'name', 'age', 'gender', 'armed', 'race', 'city', 'state',
       'flee', 'body_camera', 'signs_of_mental_illness',
       'police_departments_involved'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import pickle
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive' , force_remount=True)

# Cargar los datos procesados
data = pd.read_csv('/content/drive/My Drive/processed_shooting_data.csv')

# Definir características y objetivo
X = data[['age', 'is_male', 'armed', 'city']]
y = data['is_fatal']

# Aplicar One-Hot Encoding a las características categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = encoder.fit_transform(X[['armed', 'city']])

# Convertir el resultado de One-Hot Encoding a un DataFrame y unirlo con las otras características
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['armed', 'city']))
X = pd.concat([X[['age', 'is_male']], X_encoded_df], axis=1)

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Guardar el modelo en Google Drive
with open('/content/drive/My Drive/police_shooting_model.pkl', 'wb') as f:
    pickle.dump(model, f)


Mounted at /content/drive


In [ ]:
import pandas as pd
import pickle
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Cargar el modelo entrenado
with open('/content/drive/My Drive/police_shooting_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Cargar el lote más reciente de datos
data = pd.read_csv('/content/drive/My Drive/processed_shooting_data.csv')

# Asegurarse de que las columnas categóricas se codifiquen igual que durante el entrenamiento
# Aplicar One-Hot Encoding a las características categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Codificar los mismos datos categóricos que se utilizaron en el entrenamiento
X_categorical = data[['armed', 'city']]
X_encoded = encoder.fit_transform(X_categorical)

# Convertir el resultado de One-Hot Encoding a un DataFrame y unirlo con las otras características
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['armed', 'city']))
X = pd.concat([data[['age', 'is_male']], X_encoded_df], axis=1)

# Generar predicciones
predictions = model.predict(X)

# Guardar las predicciones en Google Drive
predicted_data = data.copy()
predicted_data['prediction'] = predictions
predicted_data.to_csv('/content/drive/My Drive/predicted_shootings.csv', index=False)


Mounted at /content/drive
